## Data Scraping

As a first step we will scrape the necessary data to proceed, namely:
- titles for each author that are publicly available
- publishing year, if it is stored in the metadata
- text strings split into single chapters

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import pandas as pd

In [3]:
from src.scraping import scrape_titles_years
from src.utils import dict_to_csv, csv_to_dict

In [4]:
authors_df = pd.read_csv('data/authors.csv')
titles_years_list = []
for url in authors_df.author_url:
    titles_years_list.extend(scrape_titles_years(url))

df = pd.DataFrame(data = titles_years_list, columns = ['author_url', 'title', 'title_url', 'year'])
df.to_csv('data/titles.csv', index=False)

In [3]:
titles_df = pd.read_csv('data/titles.csv')

In [4]:
from src.scraping import scrape_chapters

In [6]:
chapters_list = []
for title in titles_df.title_url:
    chapters_list.append(scrape_chapters(title))

df = pd.DataFrame(data = chapters_list, columns = ['title_url',
                                              'chapter_num',
                                              'chapter'])
df.to_csv('data/chapters.csv', index=False) 

## Using API's
If scraping the years from projekt-gutenberg.org does not yield any results, we'll have to rely on API's. Trying to avoid that because of hard limits though.

In [ ]:
from src.utils import PrivateKeysHandler

In [22]:
relative_path_to_file = '.env'
keys = PrivateKeysHandler(relative_path_to_file)
api_key_dict = keys.load_keys('APIs')

In [52]:
# import requests

# url = "https://google-web-search.p.rapidapi.com/"

# querystring = {"query":"theodor+fontane+effi+briest","max":"1"}

# headers = {
# 	"X-RapidAPI-Key": api_key_dict['google_web_search_key'],
# 	"X-RapidAPI-Host": "google-web-search.p.rapidapi.com"
# }

# response = requests.request("GET", url, headers=headers, params=querystring)

# print(response.text)

{"search_term":"theodor+fontane+effi+briest","search_site":null,"knowledge_panel":{"name":"Effi Briest","label":"Novel by Theodor Fontane","description":{"text":"Effi Briest is a realist novel by Theodor Fontane. Published in book form in 1895, Effi Briest marks both a watershed and a climax in the poetic realism of literature.","url":"https:\/\/en.wikipedia.org\/wiki\/Effi_Briest","site":"Wikipedia"},"image":{"url":"https:\/\/encrypted-tbn0.gstatic.com\/images?q=tbn:mP-X_DD_yDH2zM","width":181,"height":278,"page_url":"https:\/\/books.google.com\/books\/about\/Effi_Briest.html?id=OYX5JoRGQkoC&source=kp_cover"},"info":[{"title":"Originally published","labels":["1894"]},{"title":"Author","labels":["Theodor Fontane"]},{"title":"Genre","labels":["Novel"]},{"title":"Adaptations","labels":["Effi Briest (1974)","The False Step (1939)"]}]},"results":[{"position":1,"url":"https:\/\/en.wikipedia.org\/wiki\/Effi_Briest","title":"Effi Briest - Wikipedia","description":"Effi Briest is a realist nov

In [59]:
#int(response.json()['knowledge_panel']['info'][0]['labels'][0])

1894